# SQL Analysis - Pink Slip Data

In [ ]:
import sqlite3
import pandas as pd

df = pd.read_excel('pink_slips_2021_2023.xlsx')

conn = sqlite3.connect(':memory:')

# build slips table - sum item prices to get total_amount per slip
slip_totals = df.groupby('slip_number')['price'].sum().reset_index().rename(columns={'price': 'total_amount'})
slip_info = df[['slip_number', 'first_initial', 'last_name', 'phone',
                'date_received', 'due_date', 'due_time']].drop_duplicates(subset='slip_number')
slips = slip_info.merge(slip_totals, on='slip_number')

# format dates as MM/DD/YYYY strings for SQL substr queries
for col in ['date_received', 'due_date']:
    slips[col] = pd.to_datetime(slips[col]).dt.strftime('%m/%d/%Y')

slips.to_sql('slips', conn, index=False)

items = df[['slip_number', 'item_type', 'work_description', 'price']]
items.to_sql('items', conn, index=False)

print('slips')
display(pd.read_sql_query('SELECT * FROM slips LIMIT 5', conn))

print('items')
display(pd.read_sql_query('SELECT * FROM items LIMIT 5', conn))

## Revenue by Month

In [39]:
pd.read_sql_query('''
    SELECT
        substr(date_received, 7, 4) || '-' || substr(date_received, 1, 2) AS month,
        COUNT(*) AS total_slips,
        ROUND(SUM(total_amount), 2) AS revenue,
        ROUND(AVG(total_amount), 2) AS avg_slip_value
    FROM slips
    GROUP BY month
    ORDER BY month
''', conn)

,month,total_slips,revenue,avg_slip_value
0,2021-01,1937,43588.0,22.50
1,2021-02,1966,44720.0,22.75
2,2021-03,2515,54852.0,21.81
3,2021-04,3254,70186.0,21.57
4,2021-05,3335,72908.0,21.86
5,2021-06,3247,67724.0,20.86
6,2021-07,2718,56924.0,20.94
7,2021-08,2401,50832.0,21.17
8,2021-09,2232,47788.0,21.41
9,2021-10,2159,47434.0,21.97


## Top 10 Customers by Total Spend

In [29]:
pd.read_sql_query('''
    SELECT
        first_initial || '. ' || last_name AS customer,
        phone,
        COUNT(*) AS visits,
        ROUND(SUM(total_amount), 2) AS total_spent,
        ROUND(AVG(total_amount), 2) AS avg_per_visit
    FROM slips
    GROUP BY first_initial, last_name, phone
    ORDER BY total_spent DESC
    LIMIT 10
''', conn)

,customer,phone,visits,total_spent,avg_per_visit
0,G. Harmon,(980) 555-7959,21,1608.0,76.57
1,E. Ellis,(704) 555-8926,23,1525.0,66.30
2,X. Lucas,(919) 555-8729,22,1446.0,65.73
3,Q. Jones,(704) 555-5822,22,1425.0,64.77
4,J. Cannon,(704) 555-3910,18,1406.0,78.11
5,B. Wells,(336) 555-4838,17,1312.0,77.18
6,F. Hart,(980) 555-2359,20,1311.0,65.55
7,C. Nunez,(704) 555-9450,18,1306.0,72.56
8,S. Schneider,(704) 555-7905,21,1306.0,62.19
9,F. Hensley,(704) 555-6115,18,1258.0,69.89


## Item Type Breakdown

Revenue and volume by item type.

In [40]:
pd.read_sql_query('''
    SELECT
        i.item_type,
        COUNT(*) AS total_items,
        ROUND(AVG(i.price), 2) AS avg_price,
        ROUND(SUM(i.price), 2) AS total_revenue,
        ROUND(SUM(i.price) * 100.0 / (SELECT SUM(price) FROM items), 1) AS pct_of_revenue
    FROM items i
    INNER JOIN slips s ON i.slip_number = s.slip_number
    GROUP BY i.item_type
    ORDER BY total_revenue DESC
''', conn)

,item_type,total_items,avg_price,total_revenue,pct_of_revenue
0,Dress,39241,9.66,379001.0,16.3
1,Pants,42928,8.20,351891.0,15.1
2,Jacket,40187,8.51,341899.0,14.7
3,Jeans,38672,8.17,316086.0,13.6
4,Coat,30858,10.04,309945.0,13.3
5,Shirt,33462,6.91,231248.0,9.9
6,Skirt,21945,8.59,188461.0,8.1
7,Shorts,19685,6.83,134476.0,5.8
8,Other,8123,8.96,72806.0,3.1


## Most Common Alterations by Item Type

In [31]:
pd.read_sql_query('''
    SELECT
        item_type,
        work_description,
        COUNT(*) AS times_performed,
        ROUND(AVG(price), 2) AS avg_price
    FROM items
    WHERE work_description IS NOT NULL AND work_description != ''
    GROUP BY item_type, work_description
    HAVING COUNT(*) > 10
    ORDER BY item_type, times_performed DESC
''', conn)

,item_type,work_description,times_performed,avg_price
0,Coat,Hem,7545,5.13
1,Coat,Shorten sleeves,6178,6.16
2,Coat,Take in waist,5578,8.18
3,Coat,Repair,4563,9.20
4,Coat,Replace zipper,2466,16.35
...,...,...,...,...
119,Skirt,Minor fix,3980,3.38
120,Skirt,Repair,2119,9.18
121,Skirt,Resize,1751,20.48
122,Skirt,Add lining,1361,25.69


## Customer Retention Analysis

Segmenting customers by visit frequency to see which groups drive the most revenue.

In [41]:
pd.read_sql_query('''
    WITH customer_visits AS (
        SELECT
            phone,
            first_initial || '. ' || last_name AS customer,
            COUNT(*) AS visit_count,
            ROUND(SUM(total_amount), 2) AS lifetime_value
        FROM slips
        GROUP BY phone
    )
    SELECT
        CASE
            WHEN visit_count = 1 THEN 'One-time'
            WHEN visit_count BETWEEN 2 AND 4 THEN 'Repeat (2-4)'
            WHEN visit_count BETWEEN 5 AND 10 THEN 'Regular (5-10)'
            ELSE 'VIP (10+)'
        END AS customer_segment,
        COUNT(*) AS num_customers,
        ROUND(COUNT(*) * 100.0 / (SELECT COUNT(*) FROM customer_visits), 1) AS pct_of_customers,
        ROUND(SUM(lifetime_value), 2) AS total_revenue,
        ROUND(SUM(lifetime_value) * 100.0 / (SELECT SUM(lifetime_value) FROM customer_visits), 1) AS pct_of_revenue,
        ROUND(AVG(lifetime_value), 2) AS avg_lifetime_value
    FROM customer_visits
    GROUP BY customer_segment
    ORDER BY num_customers DESC
''', conn)

,customer_segment,num_customers,pct_of_customers,total_revenue,pct_of_revenue,avg_lifetime_value
0,Repeat (2-4),9072,42.4,642873.0,27.6,70.86
1,One-time,5678,26.5,141159.0,6.1,24.86
2,Regular (5-10),4779,22.3,859401.0,37.0,179.83
3,VIP (10+),1876,8.8,682380.0,29.3,363.74


## Quarterly Revenue Analysis

Comparing revenue across Q1-Q4 to check for seasonal patterns.

In [33]:
pd.read_sql_query('''
    WITH quarterly_data AS (
        SELECT
            substr(date_received, 7, 4) AS year,
            CASE
                WHEN substr(date_received, 1, 2) IN ('01', '02', '03') THEN 'Q1 (Jan-Mar)'
                WHEN substr(date_received, 1, 2) IN ('04', '05', '06') THEN 'Q2 (Apr-Jun)'
                WHEN substr(date_received, 1, 2) IN ('07', '08', '09') THEN 'Q3 (Jul-Sep)'
                ELSE 'Q4 (Oct-Dec)'
            END AS quarter,
            total_amount
        FROM slips
    )
    SELECT
        quarter,
        COUNT(*) AS total_orders,
        ROUND(SUM(total_amount), 2) AS revenue,
        ROUND(AVG(total_amount), 2) AS avg_order_value,
        ROUND(SUM(total_amount) * 100.0 / (SELECT SUM(total_amount) FROM slips), 1) AS pct_of_annual_revenue
    FROM quarterly_data
    GROUP BY quarter
    ORDER BY quarter
''', conn)

,quarter,total_orders,revenue,avg_order_value,pct_of_annual_revenue
0,Q1 (Jan-Mar),19129,501563.0,26.22,21.6
1,Q2 (Apr-Jun),29642,759979.0,25.64,32.7
2,Q3 (Jul-Sep),22231,561852.0,25.27,24.2
3,Q4 (Oct-Dec),18991,502419.0,26.46,21.6


## Revenue by Alteration Type

In [ ]:
pd.read_sql_query('''
    SELECT
        work_description AS alteration_type,
        COUNT(*) AS times_performed,
        ROUND(SUM(price), 2) AS total_revenue,
        ROUND(AVG(price), 2) AS avg_price,
        ROUND(SUM(price) * 100.0 / (SELECT SUM(price) FROM items), 1) AS pct_of_revenue
    FROM items
    WHERE work_description IS NOT NULL AND work_description != ''
    GROUP BY work_description
    ORDER BY total_revenue DESC
    LIMIT 5
''', conn)

,alteration_type,times_performed,total_revenue,avg_price,pct_of_revenue
0,Take in waist,50915,417060.0,8.19,17.9
1,Hem,79555,408603.0,5.14,17.6
2,Resize,19035,392273.0,20.61,16.9
3,Repair,29309,271501.0,9.26,11.7
4,Add lining,6803,175766.0,25.84,7.6
